In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
%cd drive/MyDrive/project-m3-notanagi/notebooks/sft_training

/content/drive/MyDrive/project-m3-notanagi/notebooks/sft_training


In [3]:
!pip install sentencepiece
!pip install -U bitsandbytes
!pip install -U git+https://github.com/huggingface/transformers.git 
!pip install -U git+https://github.com/huggingface/peft.git
!pip install -U git+https://github.com/huggingface/accelerate.git
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 57.8 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.2/92.2 MB 8.2 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-ozfpv0sg
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-ozfpv0sg
  Resolved https://github.com/huggingface/transformers.git to commit 847b47c0eed4e6ab904f584fb415e3d3a397867f
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 18.6 MB/s eta 0:00:0

In [2]:
import sys
MODULE_PATHS = ["../../modules"]
for module_path in MODULE_PATHS:
    if module_path not in sys.path:
        sys.path.append(module_path)

import json


from fine_tuning import build_finetuning_hf_dataset
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Trainer, TrainingArguments
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model
from datasets import load_dataset
from transformers import BitsAndBytesConfig


%load_ext autoreload
%autoreload 2


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 7.5
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so...


/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: /usr/lib64-nvidia did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('//172.28.0.1'), PosixPath('http'), PosixPath('8013')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('--logtostderr --listen_host=172.28.0.12 --

In [3]:
model_id = "lmsys/fastchat-t5-3b-v1.0"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForSeq2SeqLM.from_pretrained(model_id, load_in_8bit=True, device_map="auto")

In [4]:
from peft import TaskType

model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q", "v"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.SEQ_2_SEQ_LM
)
model = get_peft_model(model, config)

# print number of trainable parameters
model.print_trainable_parameters()

trainable params: 9,437,184 || all params: 2,859,120,640 || trainable%: 0.3300729555783977


In [5]:
# Verifying the datatypes.
dtypes = {}
for _, p in model.named_parameters():
    dtype = p.dtype
    if dtype not in dtypes:
        dtypes[dtype] = 0
    dtypes[dtype] += p.numel()
total = 0
for k, v in dtypes.items():
    total += v
for k, v in dtypes.items():
    print(k, v, v / total)

torch.float32 644528128 0.22542879757602674
torch.int8 2214592512 0.7745712024239733


In [15]:
max_input_length = 512
# change max input length of tokenizer to max_input_length
tokenizer.model_max_length = max_input_length
with open("../../submission_items/gen_dataset_NotAnAGI.json", "rb") as reader:
    dataset = build_finetuning_hf_dataset(json.load(reader), tokenizer=tokenizer, max_input_length=max_input_length)

Filter:   0%|          | 0/10647 [00:00<?, ? examples/s]

dropped examples:  2838


Map:   0%|          | 0/7809 [00:00<?, ? examples/s]

Map:   0%|          | 0/7809 [00:00<?, ? examples/s]

In [16]:
dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 7418
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 391
    })
})

In [17]:
dataset["train"][1]["labels"]

tensor([ 1474, 32103, 29537, 32103,    50, 32103,  1561,  1257, 32103,    20,
        32103,  5160, 32103, 12628,    15,     6, 32103,   678, 32103,    20,
         5951,     7, 32103, 29537, 32103,    90, 32103,  6053, 32103,     3,
           26,    31, 10865,     7, 32103, 18417,    15,     7, 32103,     3,
           35, 32103,  5496, 32103,    20, 32103,    50, 32103,  9989, 32103,
           20, 32103,     3,    40,    31,  8961,     5, 32103,  3039, 32103,
          197, 32103,  1990,     6, 32103,    50, 32103,  9989, 32103,    20,
        32103,     3,    40,    31,  8961, 32103,   259, 32103, 20457,   721,
        32103,   260, 32103,     3,    40,    31,    35,  3276, 32103, 13673,
        32103,  1514,    29,  3229,     5, 32103,   301,    31,   138, 11498,
          189,   526, 32103,     3, 30265, 32103,   245, 32103, 10534, 32103,
            3, 10865, 32103,   108, 32103,  1514,    29,  2423,   632,  3229,
            6, 32103,  3731,   106, 32103,     3,   173, 32103, 

In [18]:
len(dataset["train"][1]["labels"])

510

# Before training

In [20]:
def generate_example(gen_model, tokenizer, dataset, num_examples=5):
    for i in range(num_examples):
        example_input = dataset["test"][i]["input_ids"].to("cuda")
        example_label = dataset["test"][i]["labels"].to("cuda")

        output = gen_model.generate(
            input_ids=example_input.unsqueeze(0),
            max_new_tokens=1000,
            temperature=0.9,
            top_p=0.9,
            num_return_sequences=1,
            repetition_penalty=100.0

        )

        print("RESULT: ", i)
        print(tokenizer.decode(output[0], skip_special_tokens=True))
        # decode input and label
        print("INPUT: ", i)
        print(tokenizer.decode(example_input, skip_special_tokens=True))
        print("LABEL: ", i)
        print(tokenizer.decode(example_label, skip_special_tokens=True))
        print("--------------------------------------------------------")

generate_example(model, tokenizer, dataset, num_examples=5)

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:31: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")
/usr/local/lib/python3.10/dist-packages/bitsandbytes/autograd/_functions.py:318: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


<pad> Yes,  I'm  ready.



In [24]:
from transformers import DataCollatorForSeq2Seq
training_args = TrainingArguments(
    #auto_find_batch_size=True,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    num_train_epochs=1,
    learning_rate=1e-3,
    fp16=False,
    save_total_limit=4,
    logging_steps=5,
    logging_strategy="steps",
    eval_steps=100,
    evaluation_strategy="steps",
    output_dir="./outputs",
    save_strategy='steps',
    save_steps=500,
    optim="paged_adamw_8bit",
    lr_scheduler_type = 'cosine',
    warmup_ratio = 0.1,
    max_steps=1000,
)
class ConversationTrainer(Trainer):    
    def compute_loss(self, model, inputs, return_outputs=False):
        assert return_outputs == False, "we did not implement return_outputs=True in RewardTrainer.compute_loss"
        
        #print("inputs: ", inputs)
        input_ids = inputs["input_ids"]
        labels = inputs["labels"]
        #print("labels: ", labels)
        print("len inputs: ", input_ids.shape)
        print("len labels: ", labels.shape)
        loss, outputs = Trainer.compute_loss(self, model, inputs, return_outputs=True)
        print("loss: ", loss)
        print("outputs: ", outputs)
        wefwefw
        return loss



        


# we want to ignore tokenizer pad token in the loss
label_pad_token_id = -100
# Data collator
data_collator = DataCollatorForSeq2Seq(
    tokenizer,
    model=model,
    padding="max_length",
    label_pad_token_id=label_pad_token_id,
    pad_to_multiple_of=8
)


trainer = Trainer(
    model=model,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    args=training_args,
    data_collator=data_collator
)



In [25]:
#15h of training of colab if padding is "max_length" ie. 2048 in data collator 
# 3h of training if it is 512
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/bitsandbytes/autograd/_functions.py:318: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Step,Training Loss
1,1.646100
2,1.578300


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 3>:3                                                                              │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/trainer.py:1645 in train                    │
│                                                                                                  │
│   1642 │   │   inner_training_loop = find_executable_batch_size(                                 │
│   1643 │   │   │   self._inner_training_loop, self._train_batch_size, args.auto_find_batch_size  │
│   1644 │   │   )                                                                                 │
│ ❱ 1645 │   │   return inner_training_loop(                                                       │
│   1646 │   │   │   args=args,                                                                    │
│   1647 │   │   │   resume_from_checkpoint=resume_from_checkpoint,                                │
│   1648 │   │   │   trial=trial,                                                                  │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/trainer.py:1938 in _inner_training_loop     │
│                                                                                                  │
│   1935 │   │   │   │   │   self.control = self.callback_handler.on_step_begin(args, self.state,  │
│   1936 │   │   │   │                                                                             │
│   1937 │   │   │   │   with self.accelerator.accumulate(model):                                  │
│ ❱ 1938 │   │   │   │   │   tr_loss_step = self.training_step(model, inputs)                      │
│   1939 │   │   │   │                                                                             │
│   1940 │   │   │   │   if (                                                                      │
│   1941 │   │   │   │   │   args.logging_nan_inf_filter                                           │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/trainer.py:2770 in training_step            │
│                                                                                                  │
│   2767 │   │   │   with amp.scale_loss(loss, self.optimizer) as scaled_loss:                     │
│   2768 │   │   │   │   scaled_loss.backward()                                                    │
│   2769 │   │   else:                                                                             │
│ ❱ 2770 │   │   │   self.accelerator.backward(loss)                                               │
│   2771 │   │                                                                                     │
│   2772 │   │   return loss.detach() / self.args.gradient_accumulation_steps                      │
│   2773                                                                                           │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:1821 in backward               │
│                                                                                                  │
│   1818 │   │   elif self.scaler is not None:                                                     │
│   1819 │   │   │   self.scaler.scale(loss).backward(**kwargs)                                    │
│   1820 │   │   else:                                                                             │
│ ❱ 1821 │   │   │   loss.backward(**kwargs)                                                       │
│   1822 │                                                   

# After training